# XYZ, a Now Python-Friendly <br/>Geospatial Data Management Service 

<p style="color:black;font-size:18px;">
<strong>Dinu Gherman, <a href="https://here.com/">HERE Technologies</a><br/>
<a href="http://2020.geopython.net">Online Python Machine Learning Conference &amp; GeoPython 2020</a><br />Sept. 21, 2020</strong></p>

## Introduction

**HERE.com:** the location company, growing beyond automotive/logistics, 30+ years experience

**XYZ:** a new, unified, accessible, scalable, managed, open geo-database

**Components:** Hub, REST API, CLI, Studio (Web-Viewer)

**Features:** Open Source, Geospatial database/service, large volume, cloud, real-time, now Python.

**This talk:** about using XYZ programmatically from Python as intended for data scientists/analysts.

## Key Concepts

**XYZ:** Hub, Spaces with IDs, Features (geometry data) with IDs, Tags, Properties

**Spaces:** Create, read, list, update, share, delete. Plus info and stats

**Features:** Add, read, update, iterate, search, cluster (hex/quad bins), delete

**Search:** Features by ID, tag, property, bbox, tile, radius or geometry

**[XYZ-Spaces-Python](https://github.com/heremaps/xyz-spaces-python)**, short `xyzspaces`:

- wraps the XYZ Hub RESTful API
- provides a higher-level `Space` class 
- imports GeoJSON, plus CSV, GeoBuff, GPX, KML, WKT, Shapefiles, and GeoPandas dataframes

**Pro plan (supported):** Virtual spaces, feature clustering, activity log (plus more to come)

## Mini-Tutorial

This section gives a short overview of basic interaction patterns with the HERE Data Hub (based on XYZ Hub, with some pro features added).

In [ ]:
import json
import os
from IPython.display import JSON

import geojson
import pandas as pd
import requests
import turfpy.measurement
from branca.colormap import linear
from ipyleaflet import basemaps, GeoJSON, LayersControl, Map, Polygon
from ipywidgets import interact

import xyzspaces
import xyzspaces.datasets
from xyzspaces.apis import HubApi
from xyzspaces import XYZ

In [ ]:
xyz_token = os.getenv("XYZ_TOKEN")
xyz = XYZ(credentials=xyz_token)

### List spaces

In [ ]:
sp_list = xyz.spaces.list()

In [ ]:
len(sp_list)

In [ ]:
sp = sp_list[0]
JSON(sp)

### Create a space

`XYZ.spaces.new()`

In [ ]:
data1 = xyzspaces.datasets.get_countries_data()
JSON(data1)

In [ ]:
m = Map(center=[0, 0], zoom=1)
m.layout.height = "300px"
m += GeoJSON(data=data1)
m

In [ ]:
world_space = xyz.spaces.new(title="PyML & GeoPy 2020 Demo", description="Demo")

In [ ]:
world_space.info

In [ ]:
sp_id = world_space.info["id"]
sp_id

In [ ]:
world_space.get_statistics()

### Add features

`space.add_features()`

In [ ]:
features_info = world_space.add_features(features=data1)

In [ ]:
world_space.get_statistics()

### Access features

`space.iter_feature()`, `space.get_feature()`, `space.get_features()`

In [ ]:
m = Map(center=[0, 0], zoom=1)
m.layout.height = "300px"
data2 = geojson.FeatureCollection(list(world_space.iter_feature()))
m += GeoJSON(data=data2)
m

In [ ]:
JSON(data1["features"][0])

In [ ]:
JSON(data2["features"][0])

`api.get_space_feature()`, `space.get_feature()`...

In [ ]:
api = HubApi(credentials=xyz_token)
JSON(api.get_space_feature(sp_id, "AFG"))

Using more convenient abstractions:

In [ ]:
afg = world_space.get_feature("AFG")
JSON(afg)

In [ ]:
m = Map(center=[30, 70], zoom=3)
m.layout.height = "300px"
m += GeoJSON(data=afg)
m

Multiple features:

In [ ]:
JSON(world_space.get_features(["AFG", "IND"]))

### Delete space

In [ ]:
# world_space.delete()

### Update features, add tags

`space.update_feature`

In [ ]:
for feat in world_space.iter_feature():
    name = feat["properties"]["name"].lower()
    if name[0] == name[-1]:
        world_space.update_feature(feature_id=feat["id"], 
                             data=feat, 
                             add_tags=["palindromish"])

### Search features with tags

`space.search()`

In [ ]:
for feat in world_space.search(tags=["palindromish"]):
    print(feat["properties"]["name"])

### Search features in bounding box

`space.features_in_bbox()`

In [ ]:
# Features in the Southern hemishpere:
m = Map(center=[-20, 0], zoom=1)
m.layout.height = "300px"
feats = list(world_space.features_in_bbox([-180, -90, 180, 0], clip=False))
m += GeoJSON(data=geojson.FeatureCollection(feats))
m

### Search features by geometry

`space.spatial_search_geometry()`

In [ ]:
deu = world_space.get_feature("DEU")
del deu["geometry"]["bbox"]  # hack

In [ ]:
m = Map(center=[50, 9], zoom=4)
feats = list(world_space.spatial_search_geometry(data=deu["geometry"]))
m += GeoJSON(data=geojson.FeatureCollection(feats))
m += GeoJSON(data=deu, style={"color": "red", "fillOpacity": 0})
m

In [ ]:
eu_feats = list(world_space.features_in_bbox([-20, 38, 35, 70]))
list(sorted([feat["properties"]["name"] for feat in eu_feats]))

In [ ]:
df = pd.DataFrame(data=[{
    "Name": eu_feat["properties"]["name"],
    "Neighbors":
        len(list(world_space.spatial_search_geometry(data=eu_feat["geometry"]))) - 1}
    for eu_feat in eu_feats
]).sort_values(by=["Neighbors", "Name"], ascending=False, axis=0)

In [ ]:
df.head()

In [ ]:
df.index = df.Name
df.plot.bar(title="#Neighboring Countries in Europe", figsize=(10, 5), xlabel='')

### Search features by tags/parameters

`space.search()`

Now using the Microsoft US Building Footprints [dataset](https://github.com/Microsoft/USBuildingFootprints) (ca. 125 M buildings), also available in a HERE Data Hub space (ID: R4QDHvd1), used in this simple [ZIP code example](https://studio.here.com/viewer/?project_id=58604429-3919-437d-8ae4-9ee9693104d1) on [HERE Studio](https://studio.here.com).

In [ ]:
from xyzspaces.datasets import get_microsoft_buildings_space as msbs

In [ ]:
ms_space = msbs()
JSON(ms_space.get_statistics())

In [ ]:
lat, lon = 38.89759, -77.03665  # White House, USA
m = Map(center=[lat, lon], zoom=18)
feats = list(ms_space.search(tags=["postalcode@20500"]))
# feats = list(space.spatial_search(lat=lat, lon=lon, radius=100))  # same
m += GeoJSON(data=geojson.FeatureCollection(feats))
m

In [ ]:
JSON(feats[0])

Compared with original feature from https://github.com/Microsoft/USBuildingFootprints:

In [ ]:
JSON({"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-77.037293,38.897578],[-77.036459,38.897577],[-77.036459,38.897592],[-77.035855,38.897591],[-77.035856,38.897546],[-77.035778,38.897545],[-77.035778,38.897399],[-77.03549,38.897399],[-77.035489,38.897735],[-77.03556,38.897735],[-77.03556,38.897771],[-77.035783,38.897771],[-77.035783,38.897706],[-77.036228,38.897707],[-77.036228,38.897817],[-77.036439,38.897818],[-77.036438,38.897928],[-77.036619,38.897929],[-77.036619,38.89781],[-77.036853,38.89781],[-77.036853,38.897708],[-77.03695,38.897708],[-77.03695,38.897708],[-77.037851,38.897709],[-77.037851,38.89747],[-77.037929,38.89747],[-77.037929,38.897427],[-77.037815,38.897427],[-77.037816,38.897339],[-77.037294,38.897338],[-77.037293,38.897578]]]},"properties":{}})

In [ ]:
gen = ms_space.search(params={"p.city": "Washington"})
wash_dc_1000 = [next(gen) for i in range(1000)]

In [ ]:
lat, lon = 38.89759, -77.03665  # White House, USA
m = Map(center=[lat, lon], zoom=11, basemap=basemaps.CartoDB.Positron)
m += GeoJSON(data=geojson.FeatureCollection(wash_dc_1000))
m

### Search features by radius

`space.spatial_search()`

In [ ]:
lat, lon = 38.89759, -77.03665  # White House, USA
m = Map(center=[lat, lon], zoom=14, basemap=basemaps.CartoDB.Positron)
features = list(ms_space.spatial_search(lat=lat, lon=lon, radius=2000))
m += GeoJSON(data=geojson.FeatureCollection(features))
m

In [ ]:
area = turfpy.measurement.area
sizes = [area(f) for f in features]
cm = linear.Oranges_04.scale(min(sizes), max(sizes))
cm

In [ ]:
lat, lon = 38.89759, -77.03665  # White House, USA
m = Map(center=[lat, lon], zoom=14, basemap=basemaps.CartoDB.Positron)
features = list(ms_space.spatial_search(lat=lat, lon=lon, radius=2000))
m += GeoJSON(
    data=geojson.FeatureCollection(features),
    hover_style={"fillOpacity": 1},
    style_callback=lambda feat: {"weight": 2, "color": cm(area(feat))}
)
m

See also a more expanded version using building house numbers in this notebook on GitHub: [docs/notebooks/building_numbers.ipynb](https://github.com/heremaps/xyz-spaces-python/blob/master/docs/notebooks/building_numbers.ipynb).

In [ ]:
# ms_space.delete()

### Cluster all space features

`space.cluster()`

In [ ]:
xyz_pro_token = os.getenv("XYZ_PRO_TOKEN")
xyz = XYZ(credentials=xyz_pro_token)
cluster_space = xyz.spaces.new(title="Cluster Demo GeoPy", description="...")

In [ ]:
info = cluster_space.add_feature(afg)

In [ ]:
lat, lon = list(reversed(turfpy.measurement.center(afg)["geometry"]["coordinates"]))
m = Map(center=[lat, lon], zoom=4, basemap=basemaps.CartoDB.Positron)
m += GeoJSON(data=afg, name="AFG", style={"color": "red"})
m

In [ ]:
@interact(abs_res=(0, 4))
def overlay(abs_res=0):
    global m
    fc = cluster_space.cluster(
        "hexbin",
        clustering_params = {"absoluteResolution": abs_res}
    )
    lay = GeoJSON(data=fc, name=f"Hexbin {abs_res}")
    try:
        prev = [l for l in m.layers if l.name.startswith("Hexbin")][0]
        m.substitute_layer(prev, lay)
    except IndexError:
        m += lay

In [ ]:
cluster_space.delete_feature("AFG")

`space.add_features_csv()`

In [ ]:
# https://ourairports.com/data/
# https://ourairports.com/data/airports.csv
url = "https://ourairports.com/data/airports.csv"
fn = os.path.basename(url)
try:
    df = pd.read_csv(fn)
except:
    df = pd.read_csv(url)
    df.to_csv(fn, index=False)

In [ ]:
df1 = df[df.continent=="EU"]
df1 = df1[df1.iso_country!="RU"]
fn = "airports_eu.csv"
df1.to_csv(fn, index=False)

In [ ]:
m = Map(center=[50, 13], zoom=3, basemap=basemaps.CartoDB.Positron)
m += LayersControl(position="topright")
m

In [ ]:
# add many single point XYZ feature from CSV file
info = cluster_space.add_features_csv(fn, "longitude_deg", "latitude_deg", id_col="id")

In [ ]:
# calculate clustered cells
fcc = cluster_space.cluster("hexbin", clustering_params={"absoluteResolution": 2})

In [ ]:
# add hex cluster cells to map
values = [f["properties"]["aggregation"]["qty"]
          for f in fcc["features"]]
cm = linear.Oranges_04.scale(min(values), max(values))
m += GeoJSON(data=fcc,
             name="Hex Clusters",
             hover_style={"fillOpacity": 0.75},
             style_callback=lambda feat: {
                 "color": cm(feat["properties"]["aggregation"]["qty"])
             })

In [ ]:
# build one multi-point GeoJSON object
coords = [[tup.longitude_deg, tup.latitude_deg] for tup in df1.itertuples()]
mp = geojson.MultiPoint([[lon, lat] for [lon, lat] in coords])
f = geojson.Feature(geometry=mp)
f["id"] = "airports"
fc = geojson.FeatureCollection([f])

In [ ]:
m += GeoJSON(data=fc, 
             name="Airports",
             point_style={'radius': 1, "weight": 1, "fillOpacity": 1})

### Cleanup

In [ ]:
cluster_space.delete()

In [ ]:
world_space.delete()

More examples are available in the [docs/notebooks](github.com/heremaps/xyz-spaces-python) folder on GitHub.

## Not shown...

- Search by tiles
- Schema validation (pro)
- Virtual spaces (pro)
- Activity log (pro)
- Rule-based tags (pro)

## Conclusions

### Main take-aways

XYZ:

- is an Open Source geospatial cloud database/service
- is the foundation of the HERE Data Hub (commercial, free plan w/o CC)
- stores geodata features as GeoJSON, organized in *spaces*
- allows to manage, scale and filter geodata easily
- loves Python: `pip install xyzspaces`, also on conda-forge
- wants you to engage and give feedback!

### Links

- https://here.xyz
- https://github.com/heremaps/xyz-spaces-python
- https://developer.here.com/products/data-hub

### Questions?